# Appmode in Jupyter

This page demonstrates how to generate online "apps" with a Jupyter Notebook interface. With the `appmode` plugin, we can create interactive experiences without requiring coding or running a specific cell.

Markdown cells will be retained, and all code cells will be run, then hidden. The outputs of each cell will be displayed as well.

Check out a further example using an [ipyvolume](https://ipyvolume.readthedocs.io/en/latest/) to render a 3D plot: [ipyvolume_demo.ipynb](ipyvolume_demo.ipynb)

In [1]:
from __future__ import division

# Load Open FEC data
from tqdm.notebook import tqdm
pbar = tqdm(leave=False)

In [2]:
import pandas as pd

def load_df_from_openFEC(zip_url, zip_file, header_url):
   
    from io import BytesIO  
    from urllib.request import urlopen
    from zipfile import ZipFile
    
    columns = []
    # Load header data
    with urlopen(header_url) as headresp:
        df  = pd.read_csv(BytesIO(headresp.read()), sep=',', header=None)
        columns = df.iloc[0].to_list()
    
    # Load contents
    with urlopen(zip_url) as zipresp:
        with ZipFile(BytesIO(zipresp.read())) as zfile:
            df = pd.read_csv(zfile.open(zip_file), sep="|", header=None, low_memory=False, names=columns)
            return df

pbar.reset(total=3)
pbar.set_description("Loading datafiles")

donations_df = load_df_from_openFEC(
    "https://www.fec.gov/files/bulk-downloads/2020/pas220.zip", "itpas2.txt",
    "https://www.fec.gov/files/bulk-downloads/data_dictionaries/pas2_header_file.csv")
pbar.update()

cmte_df = load_df_from_openFEC(
    "https://www.fec.gov/files/bulk-downloads/2020/cm20.zip", "cm.txt",
    "https://www.fec.gov/files/bulk-downloads/data_dictionaries/cm_header_file.csv")
cmte_df = cmte_df.set_index('CMTE_ID')
pbar.update()

cand_df = load_df_from_openFEC(
    "https://www.fec.gov/files/bulk-downloads/2020/cn20.zip", "cn.txt",
    "https://www.fec.gov/files/bulk-downloads/data_dictionaries/cn_header_file.csv")
cand_df = cand_df.set_index('CAND_ID')
pbar.update()
pbar.close()

In [6]:
import ipywidgets as ipw

cb_input = ipw.Combobox(description='', options=[str(v) for v in cmte_df.CMTE_NM.tolist()], placeholder="Start typing here.")
html_output = ipw.HTML()

def on_go(btn):
    cmte_id_sel = cmte_df.index[cmte_df.CMTE_NM == cb_input.value]
    if cmte_id_sel.empty:
        html_output.value = "PAC not found"
        # TODO make popup of company not found
        return
    sc_df = get_scorecard(cmte_id_sel.format()[0])
    html_output.value = sc_df.to_html(header=False,columns=['CAND_NAME','TRANSACTION_AMT'],index=False)
    #disp_df = sc_df.reindex(columns=['CAND_NAME','TRANSACTION_AMT'])

def on_clear(btn):
    cb_input.value = ''
    html_output.value = ''
    
def get_scorecard(cmte_id):
    cmte_donations_df = donations_df[donations_df.CMTE_ID == cmte_id] # Select all donations where the CMTE_ID == cmte_id_sel

    cmte_donations_df = cmte_donations_df.groupby('CAND_ID').TRANSACTION_AMT.sum()   # Sum transactions
    cmte_donations_df = pd.DataFrame(cmte_donations_df)
    cmte_donations_df = cmte_donations_df.merge(cand_df, how='left', left_index=True, right_index=True, suffixes=('',''))   # Merge candidate data into result
    
    return cmte_donations_df
    

go_button = ipw.Button(description='Go!', layout=ipw.Layout(width="45px"))
go_button.on_click(on_go)

clear_button = ipw.Button(description='Clear', layout=ipw.Layout(width="90px"))
clear_button.on_click(on_clear)

row1 = ipw.HBox((cb_input, go_button, clear_button))
row2 = html_output
ipw.VBox((row1, row2))

0
